# SQL 

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Задания

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [1]:
#pip install psycopg2-binary

In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [4]:
query = """
    SELECT *
    FROM books
    LIMIT 5
"""
pd.io.sql.read_sql(query, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
query = """
    SELECT *
    FROM authors
    LIMIT 5
"""
pd.io.sql.read_sql(query, con = engine) 

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
query = """
    SELECT *
    FROM publishers
    LIMIT 5
"""
pd.io.sql.read_sql(query, con = engine) 

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [7]:
query = """
    SELECT *
    FROM ratings
    LIMIT 5
"""
pd.io.sql.read_sql(query, con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [8]:
query = """
    SELECT *
    FROM reviews
    LIMIT 5
"""
pd.io.sql.read_sql(query, con = engine) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Задание 1

Посчитайте, сколько книг вышло после 1 января 2000 года

In [9]:
query = """
    SELECT COUNT (book_id)
    FROM books
    WHERE CAST (publication_date AS date) > '01.01.2000'
"""
pd.io.sql.read_sql(query, con = engine) 

,count
0,819


**После 1 января 2000 года вышло 819 книг**

## Задание 2

Для каждой книги посчитайте количество обзоров и среднюю оценку

In [10]:
query = """

SELECT b.book_id,
       title,
       COUNT(DISTINCT(re.review_id)),
       AVG(r.rating)
FROM books b
LEFT JOIN reviews re ON re.book_id = b.book_id
LEFT JOIN ratings r ON b.book_id = r.book_id
GROUP BY b.book_id
ORDER BY  count DESC

"""
pd.io.sql.read_sql(query, con = engine) 

,book_id,title,count,avg
0,948,Twilight (Twilight #1),7,3.662500
1,963,Water for Elephants,6,3.977273
2,734,The Glass Castle,6,4.206897
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,808,The Natural Way to Draw,0,3.000000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,221,Essential Tales and Poems,0,4.000000


## Задание 3

Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [11]:
query = """    
WITH m AS (
            SELECT publisher,
                   COUNT(book_id) amount
            FROM books b
            JOIN publishers p ON b.publisher_id = p.publisher_id
            WHERE num_pages > 50
            GROUP BY publisher
            ORDER BY amount DESC )
 
SELECT *
FROM m
where amount = 
    (SELECT MAX(amount)
    FROM m)

    
"""
pd.io.sql.read_sql(query, con = engine) 

,publisher,amount
0,Penguin Books,42


**Издательство, выпустившее набольшее число книг толще 50 страниц - Penguin Books. Они выпустили 42 книги**

## Задание 4

Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [12]:
query = """    
WITH q AS (
            SELECT author,
                   AVG(rating) rating
            FROM authors a
            JOIN books b ON a.author_id = b.author_id
            JOIN ratings r ON b.book_id = r.book_id
            GROUP BY b.book_id, a.author_id
            HAVING COUNT(rating_id) >= 50 )
    
    SELECT author,
           AVG(rating) rating
    FROM q
    GROUP BY author
    ORDER BY rating DESC
    LIMIT 1
        

"""
pd.io.sql.read_sql(query, con = engine) 

,author,rating
0,J.K. Rowling/Mary GrandPré,4.283844


**Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré. Средняя оценка 4.3**

## Задание 5

Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [13]:
query = """    
WITH f AS (
            SELECT username,
                   COUNT(review_id) amount
            FROM reviews
            WHERE username IN ( SELECT username
                                FROM ratings
                                GROUP BY username
                                HAVING COUNT(rating_id) > 50 )
            GROUP BY username)
        
        SELECT AVG(amount)
        FROM f
"""
pd.io.sql.read_sql(query, con = engine) 

,avg
0,24.333333


**Пользователи, поставившие больше 50 оценок, в среднем пишут 24.3 обзора**

## Вывод

- После 1.01.2000 вышло 819 книг 
- Максимальное кол-во обзоров на книгу- 7 (Twilight (Twilight #1)). Для 6 книг обзоров нет. Минимильный средний рейтинг книги - 1.5, максимальный - 5. 
- Издательство, выпустившее наибольшее число книг больше 50 страниц - Penguin Books
- Автор с самой высокой средней оценкой книг (имеющих более 50 оценок) - J.K. Rowling/Mary GrandPré	- 4,3
- Среднее количество обзоров от пользователей, поставивших 50 оценок, - 24.3 
